In [33]:
# load env variables from .env file
from dotenv import load_dotenv
import os
load_dotenv()


True

In [34]:

chatgpt_wed_api_key = os.environ.get("chatgpt_wed_api_key")
chatgpt_wed_region = os.environ.get("chatgpt_wed_region")
chatgpt_wed_endpoint = os.environ.get("chatgpt_wed_endpoint")
chatgpt_wed_api_version = os.environ.get("chatgpt_wed_api_version")
chatgpt_wed_deployment = os.environ.get("chatgpt_wed_deployment")
search_wed_endpoint = os.environ.get("search_wed_endpoint")
search_wed_key = os.environ.get("search_wed_key")
search_wed_index = os.environ.get("search_wed_index")

In [35]:
from urllib.parse import urljoin, urlparse 
aoai_extension_endpoint = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgpt_wed_deployment}/extensions/chat/completions?api-version={chatgpt_wed_api_version}")
# chatgpt_api_url = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgot_wed_deployment}/completions?api-version={chatgpt_wed_api_version}")
chatgpt_api_url = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgpt_wed_deployment}/chat/completions?api-version=2023-03-15-preview")


In [36]:
print(f"AOAI Endpoint: {aoai_extension_endpoint}")
print(f"ChatGPT Endpoint: {chatgpt_api_url}")

AOAI Endpoint: https://tetetetetetete.openai.azure.com/openai/deployments/chatgpt/extensions/chat/completions?api-version=2023-06-01-preview
ChatGPT Endpoint: https://tetetetetetete.openai.azure.com/openai/deployments/chatgpt/chat/completions?api-version=2023-03-15-preview


In [37]:
import json 

def fprint(obj):
    # cast obj to dict before printing



    print(json.dumps(dict(obj), indent=4))


messages = { 
    "messages": [
        {
            "role": "system",
		    "content": """Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. 
Be brief in your answers.
Say goodbye at the end of every response. 
            """
        }, 
        {
            "role": "user", 
            "content": "Hi, I'm looking for a wedding venue in the UK. Can you help me?"
        }

    ]
}


##TODO:  Include feilds mapping here
datasources = {
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_wed_endpoint,
                "key": search_wed_key,
                "indexName": search_wed_index,
                "fieldsMapping": {
                    "titleField": "sourcefile",
                    "urlField": "sourcepage",
                    "filepathField": "sourcepage",
                    "contentFields": [
                        "content"
                    ],
                    "contentFieldsSeparator": "\n"
                },
                "inScope": "true",
                "topNDocuments": 3,
                "queryType": "semantic",
                "semanticConfiguration": "default",
                "roleInformation": ""
            }
        }
    ]
}


headers = {
    "Content-Type": "application/json",
    "api-key": f"{chatgpt_wed_api_key}",
    "chatgpt_url": chatgpt_api_url, 
    "chatgpt_key": chatgpt_wed_api_key,
}



In [38]:
print(headers.keys())

dict_keys(['Content-Type', 'api-key', 'chatgpt_url', 'chatgpt_key'])


In [39]:

# combine datasources and messages into a payload
payload = {**datasources, **messages}
#fprint(payload)


In [40]:
import requests
response = requests.post(aoai_extension_endpoint, headers=headers, data=json.dumps({**datasources, **messages}))
fprint(response.json())


{
    "id": "98e8c58a-3489-45ed-8b21-d8aca2b69366",
    "model": "gpt-35-turbo",
    "created": 1687129850,
    "object": "chat.completion",
    "choices": [
        {
            "index": 0,
            "messages": [
                {
                    "index": 0,
                    "role": "tool",
                    "content": "{\"citations\": [], \"intent\": \"Hi, I'm looking for a wedding venue in the UK. Can you help me?\"}",
                    "end_turn": false
                },
                {
                    "index": 1,
                    "role": "assistant",
                    "content": "I don't find any information related to your question from the retrieved documents. As a result, I am unable to provide an answer.",
                    "end_turn": true
                }
            ]
        }
    ]
}


In [41]:
# read response as dict 
response_dict = response.json()

response_dict["choices"][0]['messages']

for m in response_dict["choices"][0]['messages']: 
    if m["role"] == "assistant":
        messages["messages"].append(m)
    

In [42]:
fprint(messages)


{
    "messages": [
        {
            "role": "system",
            "content": "Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. \nBe brief in your answers.\nSay goodbye at the end of every response. \n            "
        },
        {
            "role": "user",
            "content": "Hi, I'm looking for a wedding venue in the UK. Can you help me?"
        },
        {
            "index": 1,
            "role": "assistant",
            "content": "I don't find any information related to your question from the retrieved documents. As a result, I am unable to provide an answer.",
            "end_turn": true
        }
    ]
}


In [43]:
messages["messages"].append({
    "role": "user",
    "content": "tell me about northwind health plan"
})

fprint(messages)

response = requests.post(aoai_extension_endpoint, headers=headers, data=json.dumps({**datasources, **messages}))


{
    "messages": [
        {
            "role": "system",
            "content": "Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. \nBe brief in your answers.\nSay goodbye at the end of every response. \n            "
        },
        {
            "role": "user",
            "content": "Hi, I'm looking for a wedding venue in the UK. Can you help me?"
        },
        {
            "index": 1,
            "role": "assistant",
            "content": "I don't find any information related to your question from the retrieved documents. As a result, I am unable to provide an answer.",
            "end_turn": true
        },
        {
            "role": "user",
            "content": "tell me about northwind health plan"
        }
    ]
}


In [44]:
len(response.json()['choices'])

1

In [45]:
fprint(response.json()['choices'][0])

{
    "index": 0,
    "messages": [
        {
            "index": 0,
            "role": "tool",
            "content": "{\"citations\": [{\"content\": \" With \\nNorthwind Health Plus, you can choose from a variety of in -network  providers, including \\nprimary care physicians, specialists, hospitals, and pharmacies. This plan also offers \\ncoverage for emergency services, both in -network and out -of-network.  \\nSUMMARY OF YOUR COSTS  \\nSUMMARY OF YOUR COSTS  \\nAt Northwind Health, we understand  that health care costs can be a burden. That\\u2019s why we \\noffer a comprehensive plan that covers the cost of medical, vision, and dental services. With \\nNorthwind Health Plus, you can choose from a variety of in -network providers, including \\nprimary care physic ians, specialists, hospitals, and pharmacies. This plan also offers \\ncoverage for emergency services, both in -network and out -of-network.  \\nYour cost for Northwind Health Plus will depend on your plan type, the s

In [46]:
# read citations from response 
citations = []
intent = []
for m in response.json()['choices'][0]['messages']: 
    if m["role"] == "tool":
        citations = json.loads(m["content"])['citations']
        intent = json.loads(m["content"])['intent']

    if m["role"] == "assistant":
        messages["messages"].append(m)

print(len(citations))

3


In [47]:
for c in citations:
    fprint(c)

{
    "content": " With \nNorthwind Health Plus, you can choose from a variety of in -network  providers, including \nprimary care physicians, specialists, hospitals, and pharmacies. This plan also offers \ncoverage for emergency services, both in -network and out -of-network.  \nSUMMARY OF YOUR COSTS  \nSUMMARY OF YOUR COSTS  \nAt Northwind Health, we understand  that health care costs can be a burden. That\u2019s why we \noffer a comprehensive plan that covers the cost of medical, vision, and dental services. With \nNorthwind Health Plus, you can choose from a variety of in -network providers, including \nprimary care physic ians, specialists, hospitals, and pharmacies. This plan also offers \ncoverage for emergency services, both in -network and out -of-network.  \nYour cost for Northwind Health Plus will depend on your plan type, the services you use, \nand the providers you visit. You can find more information about cost -sharing \narrangements on the Northwind Health website.  \n

In [48]:
print(intent)

tell me about northwind health plan
